### Step 1: Add new words

In [ ]:
import json, time, random, datetime
import IPython
import pandas as pd

db = pd.read_csv("oxford3000-5000.db")
with open("oxford3000-5000.note") as f:
    note = json.load(f)

old_index = set()
for item in note:
    old_index.add(item["index"])
new_index = set(db["index"]) - old_index
if len(new_index) >= 10: # Learn only 10 new words each time
    ten_new_words = db[db["index"].isin(random.sample(new_index, 10))]
    for key, value in ten_new_words.iterrows():
        IPython.display.clear_output()
        time.sleep(0.5)
        out = ""
        out += "Word: {}\n".format(value["word"])
        out += "Oxford 3000: {}\n".format(value["ox3000"])
        out += "Oxford 5000: {}\n".format(value["ox5000"])
        out += "POS: {}\n".format(value["pos"])
        out += "Belong to: {}\n".format(value["belong_to"])
        out += "Definition: {}\n".format(value["definition"])
        out += "Input note: "
        # hint: |   to split meanings
        #       <>  to get examples
        #       ___ to hide words
        hint = input(out)
        note.append({"index": value[0],
                     "note": hint,
                     "examination": str(datetime.datetime.now().date()), # examination date
                     "duration": 0,                                      # duration between two consecutive exams
                     "total": 0,                                         # number of examination times
                     "stability": 0})                                    # number of correct times in a row
    with open("oxford3000-5000.note", "w") as f:
        note.sort(key=lambda x: x["stability"])
        json.dump(note, f)
else:
    print("Not sufficient 10 new words")

### Step 2: Examination

In [ ]:
import json, time, datetime
import IPython
import numpy as np
import pandas as pd

db = pd.read_csv("oxford3000-5000.db")
with open("oxford3000-5000.note") as f:
    note = json.load(f)
    
def perform_test(key):
    IPython.display.clear_output()
    time.sleep(0.5)
    subvalue = note[key]
    value = db.loc[db[db["index"] == subvalue["index"]].index[0], :]
    out = ""
    out += "Oxford 3000: {}\n".format(value["ox3000"])
    out += "Oxford 5000: {}\n".format(value["ox5000"])
    out += "POS: {}\n".format(value["pos"])
    out += "Belong to: {}\n".format(value["belong_to"])
    out += "Note: {}\n".format(subvalue["note"])
    out += "Input answer: "
    answer = input(out)
    correct = ""
    while correct not in ["yes", "no"]:
        out = ""
        out += "Word:         {}\n".format(value["word"])
        out += "Definition: {}\n".format(value["definition"])
        out += "So, is your answer correct? (yes/no) "
        correct = input(out)
    return correct == "yes"
    
bad = set()
normal = set()
good = set()
perfect = set()
for i in range(len(note)):
    if datetime.datetime.now().date() >= datetime.date.fromisoformat(note[i]["examination"]):
        bad.add(i)
    if len(bad) + len(normal) + len(good) == 10:
        while len(perfect) == 0:
            new_bad = set()
            new_normal = set()
            new_good = set()
            new_perfect = set()
            for key in bad:
                if perform_test(key):
                    new_normal.add(key)
                else:
                    new_bad.add(key)
                    note[key]["duration"] = 0
                    note[key]["stability"] = 0
            for key in normal:
                if perform_test(key):
                    new_good.add(key)
                else:
                    new_bad.add(key)
                    note[key]["duration"] = 0
                    note[key]["stability"] = 0
            for key in good:
                if perform_test(key):
                    new_perfect.add(key)
                else:
                    new_bad.add(key)
                    note[key]["duration"] = 0
                    note[key]["stability"] = 0
            bad = new_bad
            normal = new_normal
            good = new_good
            perfect = new_perfect
        for key in perfect:
            note[key]["total"] += 1
            note[key]["stability"] += 1
            note[key]["duration"] += 1 + int(note[key]["total"] / (1 + np.exp(-note[key]["stability"])))
            note[key]["examination"] = str(datetime.datetime.now().date() +
                                           datetime.timedelta(days = note[key]["duration"]))
        perfect = set()
        with open("oxford3000-5000.note", "w") as f:
            json.dump(note, f)

### Utilities

In [ ]:
import json, datetime

with open("oxford3000-5000.note") as f:
    note = json.load(f)

cnt = 0
total = 0
for i in range(len(note)):
    total += 1
    if datetime.datetime.now().date() >= datetime.date.fromisoformat(note[i]["examination"]):
        cnt += 1
print("{}/{}".format(cnt, total))